In [ ]:
!pip install fastai==1.0.61

## Import libraries

In [ ]:
import os
import boto3
from earthai.ml import *
import statistics
import glob

from fastai import *
from fastai.vision import *

## Download final models and .tar file from S3 bucket and extract contents

In [ ]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('sfi-shared-assets')

In [ ]:
def download_model(MODEL_PATH):
    if not os.path.exists(LOCAL_DIR + MODEL_PATH.split("/")[-1].replace(".pkl", "")):
        os.makedirs(LOCAL_DIR + MODEL_PATH.split("/")[-1].replace(".pkl", ""))
    bucket.download_file(MODEL_PATH, LOCAL_DIR + MODEL_PATH.split("/")[-1].replace(".pkl", "") + "/export.pkl")

In [ ]:
def get_classwise_counts(items, classes):
    series = pd.value_counts(items).sort_index()
    series.index = classes
    
    return series

In [ ]:
def get_mode(p1, p2, p3):
    try:
        return statistics.mode([str(p1[0]), str(p2[0]), str(p3[0])])
    except:
        mx = max(p1[2].max(), p2[2].max(), p3[2].max())
        if mx == p1[2].max():
            return str(p1[0])
        elif mx == p2[2].max():
            return str(p2[0])
        elif mx == p3[2].max():
            return str(p3[0])

## Read in image data

In [ ]:
DATA_PATH = 'S2-RGB-macro-localization-model-build4/ALD_S2_RGB_chips_v4p1_train4.tar'
LOCAL_DIR = '/scratch/'

In [ ]:
bucket.download_file(DATA_PATH, LOCAL_DIR + DATA_PATH.split("/")[-1])
unix_code = 'tar -C /scratch/ -xf '+LOCAL_DIR + DATA_PATH.split("/")[-1]
os.system(unix_code)

In [ ]:
data = (ImageDataBunch.from_folder(LOCAL_DIR + DATA_PATH.split("/")[-1].replace(".tar", ""), train='train', valid='validate', 
                                   bs=16, num_workers=0, seed=42)
        .normalize(imagenet_stats))

### Display class-wise counts for training and validation sets.

In [ ]:
for subset, label in zip((data.train_ds, data.valid_ds), ('Training set', 'Validation set')):
    print('--- {} ---'.format(label))
    print(get_classwise_counts(subset.y.items, subset.classes))

## Multiclass Models - Download

In [ ]:
DENSENET161_MODEL_PATH = 'S2-RGB-macro-localization-model-build4/S2-RGB-model-results4/densenet161_multiclass_final.pkl'
RESNET50_MODEL_PATH = 'S2-RGB-macro-localization-model-build4/S2-RGB-model-results4/resnet50_multiclass_final.pkl'
VGG13_MODEL_PATH = 'S2-RGB-macro-localization-model-build4/S2-RGB-model-results4/vgg13_multiclass_final.pkl'

In [ ]:
download_model(DENSENET161_MODEL_PATH)
download_model(RESNET50_MODEL_PATH)
download_model(VGG13_MODEL_PATH)

In [ ]:
densenet161_multi_model = load_learner(LOCAL_DIR + DENSENET161_MODEL_PATH.split("/")[-1].replace(".pkl", ""))
resnet50_multi_model = load_learner(LOCAL_DIR + RESNET50_MODEL_PATH.split("/")[-1].replace(".pkl", ""))
vgg13_multi_model = load_learner(LOCAL_DIR + VGG13_MODEL_PATH.split("/")[-1].replace(".pkl", ""))

## Cement Binary Models - Download

In [ ]:
DENSENET161_MODEL_PATH = 'S2-RGB-macro-localization-model-build4/S2-RGB-model-results4/densenet161_cement_binary_final.pkl'
RESNET50_MODEL_PATH = 'S2-RGB-macro-localization-model-build4/S2-RGB-model-results4/resnet50_cement_binary_final.pkl'
VGG13_MODEL_PATH = 'S2-RGB-macro-localization-model-build4/S2-RGB-model-results4/vgg13_cement_binary_final.pkl'

In [ ]:
download_model(DENSENET161_MODEL_PATH)
download_model(RESNET50_MODEL_PATH)
download_model(VGG13_MODEL_PATH)

In [ ]:
densenet161_cement_model = load_learner(LOCAL_DIR + DENSENET161_MODEL_PATH.split("/")[-1].replace(".pkl", ""))
resnet50_cement_model = load_learner(LOCAL_DIR + RESNET50_MODEL_PATH.split("/")[-1].replace(".pkl", ""))
vgg13_cement_model = load_learner(LOCAL_DIR + VGG13_MODEL_PATH.split("/")[-1].replace(".pkl", ""))

## Steel Binary Models - Download

In [ ]:
DENSENET161_MODEL_PATH = 'S2-RGB-macro-localization-model-build4/S2-RGB-model-results4/densenet161_steel_binary_final.pkl'
RESNET50_MODEL_PATH = 'S2-RGB-macro-localization-model-build4/S2-RGB-model-results4/resnet50_steel_binary_final.pkl'
VGG13_MODEL_PATH = 'S2-RGB-macro-localization-model-build4/S2-RGB-model-results4/vgg13_steel_binary_final.pkl'

In [ ]:
download_model(DENSENET161_MODEL_PATH)
download_model(RESNET50_MODEL_PATH)
download_model(VGG13_MODEL_PATH)

In [ ]:
densenet161_steel_model = load_learner(LOCAL_DIR + DENSENET161_MODEL_PATH.split("/")[-1].replace(".pkl", ""))
resnet50_steel_model = load_learner(LOCAL_DIR + RESNET50_MODEL_PATH.split("/")[-1].replace(".pkl", ""))
vgg13_steel_model = load_learner(LOCAL_DIR + VGG13_MODEL_PATH.split("/")[-1].replace(".pkl", ""))

## Score models on validation sets

In [ ]:
# Multiclass results
valid_preds_densenet161_multi = []
valid_preds_resnet50_multi = []
valid_preds_vgg13_multi = []
valid_preds_ensemble_multi = []
valid_y_multi = []

# Cement binary results
valid_preds_densenet161_cement = []
valid_preds_resnet50_cement = []
valid_preds_vgg13_cement = []
valid_preds_ensemble_cement = []
valid_y_cement = []

# Steel binary results
valid_preds_densenet161_steel = []
valid_preds_resnet50_steel = []
valid_preds_vgg13_steel = []
valid_preds_ensemble_steel = []
valid_y_steel = []

for i in range(0, len(data.valid_ds.x)):
    
    # Multiclass
    valid_y_multi.append(str(data.valid_ds.y[i]))
    p1 = densenet161_multi_model.predict(data.valid_ds.x[i])
    valid_preds_densenet161_multi.append(str(p1[0]))
    p2 = resnet50_multi_model.predict(data.valid_ds.x[i])
    valid_preds_resnet50_multi.append(str(p2[0]))
    p3 = vgg13_multi_model.predict(data.valid_ds.x[i])
    valid_preds_vgg13_multi.append(str(p3[0]))
    valid_preds_ensemble_multi.append(get_mode(p1, p2, p3))
    
    # Cement binary
    valid_y_cement.append(str(data.valid_ds.y[i]))
    p1 = densenet161_cement_model.predict(data.valid_ds.x[i])
    valid_preds_densenet161_cement.append(str(p1[0]))
    p2 = resnet50_cement_model.predict(data.valid_ds.x[i])
    valid_preds_resnet50_cement.append(str(p2[0]))
    p3 = vgg13_cement_model.predict(data.valid_ds.x[i])
    valid_preds_vgg13_cement.append(str(p3[0]))
    valid_preds_ensemble_cement.append(get_mode(p1, p2, p3))
    
    # Steel binary
    valid_y_steel.append(str(data.valid_ds.y[i]))
    p1 = densenet161_steel_model.predict(data.valid_ds.x[i])
    valid_preds_densenet161_steel.append(str(p1[0]))
    p2 = resnet50_steel_model.predict(data.valid_ds.x[i])
    valid_preds_resnet50_steel.append(str(p2[0]))
    p3 = vgg13_steel_model.predict(data.valid_ds.x[i])
    valid_preds_vgg13_steel.append(str(p3[0]))
    valid_preds_ensemble_steel.append(get_mode(p1, p2, p3))
    
    print('Done with ', i+1, ' out of ', len(data.valid_ds.x))

## Print results

### DenseNet161 MultiClass Model Results

In [ ]:
print("\033[1m\033[4m" + "VALIDATION RESULTS:" + "\033[0m\n")

showConfusionMatrix(np.array(valid_preds_densenet161_multi), np.array(valid_y_multi))
printOverallStats(np.array(valid_preds_densenet161_multi), np.array(valid_y_multi))
printClassStats(np.array(valid_preds_densenet161_multi), np.array(valid_y_multi))

### ResNet50 MultiClass Model Results

In [ ]:
print("\033[1m\033[4m" + "VALIDATION RESULTS:" + "\033[0m\n")

showConfusionMatrix(np.array(valid_preds_resnet50_multi), np.array(valid_y_multi))
printOverallStats(np.array(valid_preds_resnet50_multi), np.array(valid_y_multi))
printClassStats(np.array(valid_preds_resnet50_multi), np.array(valid_y_multi))

### VGG13 MultiClass Model Results

In [ ]:
print("\033[1m\033[4m" + "VALIDATION RESULTS:" + "\033[0m\n")

showConfusionMatrix(np.array(valid_preds_vgg13_multi), np.array(valid_y_multi))
printOverallStats(np.array(valid_preds_vgg13_multi), np.array(valid_y_multi))
printClassStats(np.array(valid_preds_vgg13_multi), np.array(valid_y_multi))

### Ensemble MultiClass Model Results

In [ ]:
print("\033[1m\033[4m" + "VALIDATION RESULTS:" + "\033[0m\n")

showConfusionMatrix(np.array(valid_preds_ensemble_multi), np.array(valid_y_multi))
printOverallStats(np.array(valid_preds_ensemble_multi), np.array(valid_y_multi))
printClassStats(np.array(valid_preds_ensemble_multi), np.array(valid_y_multi))

### DenseNet161 Cement Binary Model Results

In [ ]:
valid_y_cement = ['landcover' if y == 'steel' else y for y in valid_y_cement]

In [ ]:
print("\033[1m\033[4m" + "VALIDATION RESULTS:" + "\033[0m\n")

showConfusionMatrix(np.array(valid_preds_densenet161_cement), np.array(valid_y_cement))
printOverallStats(np.array(valid_preds_densenet161_cement), np.array(valid_y_cement))
printClassStats(np.array(valid_preds_densenet161_cement), np.array(valid_y_cement))

### ResNet50 Cement Binary Model Results

In [ ]:
print("\033[1m\033[4m" + "VALIDATION RESULTS:" + "\033[0m\n")

showConfusionMatrix(np.array(valid_preds_resnet50_cement), np.array(valid_y_cement))
printOverallStats(np.array(valid_preds_resnet50_cement), np.array(valid_y_cement))
printClassStats(np.array(valid_preds_resnet50_cement), np.array(valid_y_cement))

### VGG13 Cement Binary Model Results

In [ ]:
print("\033[1m\033[4m" + "VALIDATION RESULTS:" + "\033[0m\n")

showConfusionMatrix(np.array(valid_preds_vgg13_cement), np.array(valid_y_cement))
printOverallStats(np.array(valid_preds_vgg13_cement), np.array(valid_y_cement))
printClassStats(np.array(valid_preds_vgg13_cement), np.array(valid_y_cement))

### Ensemble Cement Binary Model Results

In [ ]:
print("\033[1m\033[4m" + "VALIDATION RESULTS:" + "\033[0m\n")

showConfusionMatrix(np.array(valid_preds_ensemble_cement), np.array(valid_y_cement))
printOverallStats(np.array(valid_preds_ensemble_cement), np.array(valid_y_cement))
printClassStats(np.array(valid_preds_ensemble_cement), np.array(valid_y_cement))

### DenseNet161 Steel Binary Model Results

In [ ]:
valid_y_steel = ['landcover' if y == 'cement' else y for y in valid_y_steel]

In [ ]:
print("\033[1m\033[4m" + "VALIDATION RESULTS:" + "\033[0m\n")

showConfusionMatrix(np.array(valid_preds_densenet161_steel), np.array(valid_y_steel))
printOverallStats(np.array(valid_preds_densenet161_steel), np.array(valid_y_steel))
printClassStats(np.array(valid_preds_densenet161_steel), np.array(valid_y_steel))

### ResNet50 Steel Binary Model Results

In [ ]:
print("\033[1m\033[4m" + "VALIDATION RESULTS:" + "\033[0m\n")

showConfusionMatrix(np.array(valid_preds_resnet50_steel), np.array(valid_y_steel))
printOverallStats(np.array(valid_preds_resnet50_steel), np.array(valid_y_steel))
printClassStats(np.array(valid_preds_resnet50_steel), np.array(valid_y_steel))

### VGG13 Steel Binary Model Results

In [ ]:
print("\033[1m\033[4m" + "VALIDATION RESULTS:" + "\033[0m\n")

showConfusionMatrix(np.array(valid_preds_vgg13_steel), np.array(valid_y_steel))
printOverallStats(np.array(valid_preds_vgg13_steel), np.array(valid_y_steel))
printClassStats(np.array(valid_preds_vgg13_steel), np.array(valid_y_steel))

### Ensemble Steel Binary Model Results

In [ ]:
print("\033[1m\033[4m" + "VALIDATION RESULTS:" + "\033[0m\n")

showConfusionMatrix(np.array(valid_preds_ensemble_steel), np.array(valid_y_steel))
printOverallStats(np.array(valid_preds_ensemble_steel), np.array(valid_y_steel))
printClassStats(np.array(valid_preds_ensemble_steel), np.array(valid_y_steel))